In [1]:
#Random circuit generation
# !pip install qiskit

In [2]:
from qiskit import *
from random import *
import numpy as np

# Random Circuit Generation (Previous Work, kindly scroll down for updated version)

The code below is for generating random circuits for further analysis. I have defined two functions - one for using width + number of gates, and other for width + depth.

I think number of gates is a more useful parameter, but I got both done here.

In [3]:
def random_cirq_number_gates(number_of_gates,width):
    qc = QuantumCircuit(width)
    list_of_gates = [(qc.h,1),(qc.z,1),(qc.cz,2),(qc.ccz,3)]
    counts = {}
    #number_of_gates = 100
    for i in range(number_of_gates):
        gate,arg = choice(list_of_gates)
        inp = []
        while len(inp) < arg:
            val = randint(0,width-1)
            if val not in inp:
                inp.append(val)
        #print(gate,inp)
        gate(*inp)
        try: counts[gate] +=1
        except KeyError: counts[gate] = 1

    return qc


In [4]:
%%time
qc = random_cirq_number_gates(400,25)
print(qc.depth())
# qc.draw(output="text", fold=-1, justify='left')

73
CPU times: total: 0 ns
Wall time: 13.5 ms


In [5]:
def random_cirq_depth(depth,width):
    qc = QuantumCircuit(width)
    list_of_gates = [(qc.h,1),(qc.z,1),(qc.cz,2),(qc.ccz,3)]
    counts = {}
    number_of_gates = 100
 #   for i in range(number_of_gates):
    while qc.depth()<depth+1:
        gate,arg = choice(list_of_gates)

        inp = []
        # print(gate,arg,inp)
        while len(inp) < arg:
            val = randint(0,width-1)
            if val not in inp:
                inp.append(val)
                # print(inp)
        # print(gate,inp)
        gate(*inp)
        try: counts[gate] +=1
        except KeyError: counts[gate] = 1

        if qc.depth() == depth+1:
            del qc.data[-1]
            break #Not deleting this will cause an infinite circuit.
    return qc


In [3]:
# qc_ = random_cirq_number_gates(20,5)

qc_ = QuantumCircuit(2)
qc_.h(0)
qc_.h(1)
qc_.cz(0,1)
qc_.h(1)




In [4]:
print(qc_.depth())
# qc_.draw(output="text", fold=-1, justify='left')

3


# Coming to circuit polynomials

Now, to convert to circuit polynomials, we can either use the circuit visualisation or alternatively the data from the circuit object directly.

## What I've Done

As you can see below, it is easy to extract the instructions to build the circuit i.e. the gates in sequential order. This can be used to get the circuit polynomial in a very systematic manner.

In [5]:
for index, instruction in enumerate(qc_.data):
    print(index, instruction.operation.name, [ qc_.find_bit(q).index for q in instruction.qubits])

0 h [0]
1 h [1]
2 cz [0, 1]
3 h [1]


In [6]:
qc_.count_ops()

OrderedDict([('h', 3), ('cz', 1)])

In [7]:
%%time
'''
Approach: We use the list of instruction operators and track a few things -

1. Current active variable in each wire
2. The polynomial term corresponding.


'''
width = qc_.width()

hcount = list(instruction.operation.name for index, instruction in enumerate(qc_.data)).count('h')
list_ = [(instruction.operation.name, [ qc_.find_bit(q).index for q in instruction.qubits]) for index, instruction in enumerate(qc_.data)]

# number_of_variables = width + hcount
wire_array = [[i] for i in range(width)]
max_new_var = width
terms = []

for entry in list_:
    gate = entry[0]
    elements = entry[1]

    if gate == 'h':
        wire_array[elements[0]].append(max_new_var)
        #print(wire_array[elements[0]])
        max_new_var+=1
        terms.append([wire_array[elements[0]][-1],wire_array[elements[0]][-2]])


    else:
        terms.append([wire_array[j][-1] for j in elements])

for term in terms:
    term.sort()
    for j in term:
        print('x'+str(j),end='')
    print(' +',end=' ')
print('\n')

print(terms)
print(max_new_var)

x0x2 + x1x3 + x2x3 + x3x4 + 

[[0, 2], [1, 3], [2, 3], [3, 4]]
5
CPU times: total: 0 ns
Wall time: 0 ns


In [8]:
wire_array

[[0, 2], [1, 3, 4]]

In [9]:
#Here I am gonna try to write a code to evaluate the terms of polynomial for given input vector.
#x = [bool(i) for i in [1,1,1,1,1]]

#x = [0 for i in range(max_new_var)]

def polyval(terms,x):#now x is an integer input
    x_bin = bin(x)[2:].zfill(max_new_var)
    val_out = bool(0)
    for term in terms:
        v = bool(1)
        for j in term:
            v &= int(x_bin[j])
        val_out ^= v

    return val_out

polyval(terms,0)

def truthtable(terms,max_new_var):
    x_num = 2**max_new_var #number of x values possible:
    tt = np.empty((x_num,1),dtype='int8')

    for i in range(x_num):

        tt[i] = polyval(terms,i)

    return tt

In [10]:
%%time
ttb = truthtable(terms,max_new_var)
#for r in ttb: print(r)
#ttb[0:2**(max_new_var - len(ivs))]

CPU times: total: 0 ns
Wall time: 985 µs


In [11]:
print(*ttb, sep=" ")

[0] [0] [0] [1] [0] [0] [1] [0] [0] [0] [1] [0] [0] [0] [0] [1] [0] [0] [0] [1] [1] [1] [0] [1] [0] [0] [1] [0] [1] [1] [1] [0]


In [13]:
ivs = [j[0] for j in wire_array]
ovs = [j[-1] for j in wire_array]

#Sifting:

def sift_inp(initial_val_int,ttb):
    #ivs = [j[0] for j in wire_array]
    group_size = 2**(max_new_var - len(ivs))
    return group_size*(initial_val_int), ttb[group_size*(initial_val_int):group_size*(initial_val_int+1), :]



sift_inp(0,ttb)

(0,
 array([[0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0]], dtype=int8))

In [14]:
ivs,ovs

([0, 1], [2, 4])

In [19]:
#amplitudes:

def statevector_(initial_val_int,ttb):
    #ovs = [j[-1] for j in wire_array]
    stvector = []
    #starting_index, sifted_table = sift_inp(initial_val_int,ttb)

    group_size = 2**(max_new_var - len(ivs))
    starting_index = group_size*(initial_val_int)

    '''

    for i in range(2**len(ovs)): #In this we scroll through all of the possible values!

        bitval  = bin(i)[2:].zfill(len(ovs))
        s = []

        for k in range(starting_index,starting_index+group_size):
            indexval = k+starting_index
            chosenbits = "".join([(bin(k)[2:].zfill((max_new_var)))[j] for j in ovs])
            if chosenbits == bitval:
                s.append(ttb[k])
        #print(1-2*np.array(s))

        '''

    s = np.zeros(2**len(ovs),dtype=int)
    for k in range(starting_index,starting_index+group_size):
        chosenbits = "".join([(bin(k)[2:].zfill(max_new_var))[j] for j in ovs])
        chosen_int = int(chosenbits,2)
        s[chosen_int] += (1-2*ttb[k,0])




 
    #s_sum = np.sum(1-2*np.array(s,dtype='int'))

        #print(bitval,':',( s_sum / ((2**.5)**(max_new_var - len(ivs)) )))
    #stvector.append( s_sum / (2**.5)**(max_new_var - len(ivs)))

    stvector = s / (2**.5)**(max_new_var - len(ivs))

    return stvector

# state_vector = [el for el in statevector_(0,ttb) if el!=0]
state_vector = statevector_(2, ttb)
print(*state_vector, sep=' ')


0.7071067811865474 0.0 0.0 -0.7071067811865474


In [21]:
ttb[3,0]

1

# IMP

I was able to build all components of this simulator (yet to add a histogram but the statevector thing is done). below, I have defined a single function to display the results.
I feel we can structure the code a bit more to return an object which is easier to work with.

In [22]:


def simulate_viapoly(qc):

    #part 1: obtaining the polynomial
    width = qc_.width()

    from collections import Counter
    hcount = list(instruction.operation.name for index, instruction in enumerate(qc_.data)).count('h')
    list_ = [(instruction.operation.name, [ qc_.find_bit(q).index for q in instruction.qubits]) for index, instruction in enumerate(qc_.data)]

    number_of_variables = width + hcount
    wire_array = [[i] for i in range(width)]
    max_new_var = width
    terms = []

    for entry in list_:
        gate = entry[0]
        elements = entry[1]

        if gate == 'h':
            wire_array[elements[0]].append(max_new_var)
            #print(wire_array[elements[0]])
            max_new_var+=1
            terms.append([wire_array[elements[0]][-1],wire_array[elements[0]][-2]])


        else:
            terms.append([wire_array[j][-1] for j in elements])

    for term in terms:
        term.sort()
        '''for j in term:
            print('x'+str(j),end='')
        print(' +',end=' ')
    print('\n')'''

    #print(terms)

    #part 2: getting the truthtable

    ttb = truthtable(terms,max_new_var)

    #part 3: obtaining which variables are input and output variables
    ivs = [j[0] for j in wire_array]
    ovs = [j[-1] for j in wire_array]

    #print('\n')
    #part 4: statevector obtaining for all possible initialised states:
    #First, we try to see all possible values of initialised states:
    instates = range(2**len(ivs))
    '''
    for instate in instates:
        print('For initial state ',instate,':')
        sv = statevector_(instate,ttb)
        print(sv,'\n')
    '''
    results = {instate:statevector_(instate,ttb) for instate in instates}
    return results


#### Polynomil Simulator

In [24]:
# qc_ = random_cirq_number_gates(200, 15)

In [25]:
%%time
res = simulate_viapoly(qc_)

CPU times: total: 0 ns
Wall time: 996 µs


#### DDSIM

In [26]:
from mqt import ddsim

In [27]:
%%time 
backend = ddsim.DDSIMProvider().get_backend("qasm_simulator")

job = backend.run(qc_, shots=10000)
counts = job.result().get_counts(qc_)
# print(counts)

CPU times: total: 0 ns
Wall time: 25 ms


I was able to basically simulate the circuit without having to define a vector storing the boolean in ANF form. However, the vector form calculation is still there below.
## Vector THingy (hiddenfornow)

This approach returns - which terms are there. Now defining a vector correspoing to the polynomial is a bit tricky (not really tricky but thoda tiring) since the mapping is v v confusing for large number of terms, so I have defined a reference vector to get the coordinate. Appropriate zero padding is done in a later step.

In [28]:
#Note: I am defining a reference vector:
#[i] + [{i,j} for every possible {i,j}] + [{i,j,k} ]

l1 = [[i] for i in range(max_new_var)]
l2 = [[j,i] for i in range(max_new_var) for j in range(i)]
l2.sort()
l3 = [[k,j,i] for i in range(max_new_var) for j in range(i) for k in range(j)]
l3.sort()

lref = l1+l2+l3
for j in range(len(lref)):
    print(j,':',lref[j])



0 : [0]
1 : [1]
2 : [2]
3 : [3]
4 : [4]
5 : [0, 1]
6 : [0, 2]
7 : [0, 3]
8 : [0, 4]
9 : [1, 2]
10 : [1, 3]
11 : [1, 4]
12 : [2, 3]
13 : [2, 4]
14 : [3, 4]
15 : [0, 1, 2]
16 : [0, 1, 3]
17 : [0, 1, 4]
18 : [0, 2, 3]
19 : [0, 2, 4]
20 : [0, 3, 4]
21 : [1, 2, 3]
22 : [1, 2, 4]
23 : [1, 3, 4]
24 : [2, 3, 4]


In [29]:
def pos_2(i,j):
    return max_new_var + 0.5*i*(2*max_new_var-i-1) + (j-i-1)

def pos_3(i,j,k):
    return (max_new_var + max_new_var*(max_new_var-1)/2) + 0.5*i*((i+1)*(2*i+1)/6 - (2*max_new_var-1)*(i+1)/2 + max_new_var*(max_new_var-1)) +0.5*(j-i-1)*(2*max_new_var-j-i-2)+(k-j-1)
pos_3(0,2,4)

def polyterm_index(term):
    if len(term) == 3:
        i,j,k = term
        pos = (max_new_var + max_new_var*(max_new_var-1)/2) + 0.5*i*((i+1)*(2*i+1)/6 - (2*max_new_var-1)*(i+1)/2 + max_new_var*(max_new_var-1)) +0.5*(j-i-1)*(2*max_new_var-j-i-2)+(k-j-1)
    elif len(term) == 2:
        i,j = term
        pos = max_new_var + 0.5*i*(2*max_new_var-i-1) + (j-i-1)
    elif len(term) == 1:
        i = term[0]
        pos = i
    return int(pos)



In [30]:
l_length = int(max_new_var*(max_new_var**2 + 5)/6 )
l_pol_short = np.zeros(l_length,dtype=int)
for t in terms:
    in_ = polyterm_index(t)
    if l_pol_short[in_]: l_pol_short[in_]=0
    else: l_pol_short[in_]=1
l_pol_short

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0])

In [31]:
from math import *
l_pol = np.array([terms.count(i)%2 for i in lref])
#for i in range(len(l_pol)):
#    print(lref[i],':',l_pol[i])

In [32]:
polyvector_big = np.pad(l_pol, (1, 2**(max_new_var)-len(l_pol)-1), 'constant')

###### l_pol

In [33]:
print(len(polyvector_big))
polyvector_big

32


array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

# Output from qiskit statevector simulator


In [34]:

from qiskit_aer import StatevectorSimulator
backend = StatevectorSimulator()
res = backend.run(qc_).result()
print(res.get_statevector())
print(res.get_counts())



Statevector([0.70710678-4.32978028e-17j, 0.        +4.32978028e-17j,
             0.        +4.32978028e-17j, 0.70710678-4.32978028e-17j],
            dims=(2, 2))
{'00': 0.5, '11': 0.5}


In [ ]:
!pip install qiskit_aer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 33.2 MB/s eta 0:00:00


In [ ]:
qc_.draw()

┌───┐        
q_0: ┤ H ├─■──────
     ├───┤ │ ┌───┐
q_1: ┤ H ├─■─┤ H ├
     └───┘   └───┘

# Random Circuit Generation

The below code generates a random circuit with the following parameters:
1. Width = Number of Qubits/Wires
2. Depth = Number of layers of simultaneously operating gates
3. HGates = (Cap on) Number of Hadamard Gates

##### [NOTE: There are chances of number of hadamards coming out to be lower. It is currently challenging to exactly monitor the number of hadamard gates for us but we can regulate it by fixing a cap.]

    
    

In [ ]:
def rnd_cg(width,depth,hgates=None):

    if hgates==None: hgates=width*depth


    qc = QuantumCircuit(width)

    list_of_gates = [(qc.h,1),(qc.z,1),(qc.cz,2),(qc.ccz,3)]
    if width<=2: del(list_of_gates[-1])
    if width<=1: del(list_of_gates[-1])
    #counts = {}
    lasgate = [None for i in range(width)]


    hcounter=0
 #   for i in range(number_of_gates):


    while qc.depth()<depth+1:
        gate,arg = choice(list_of_gates)
        inp = []

        #print(width,arg)

        inp = sample(range(0,width),arg)
        inp.sort()
        #print(gate,inp)

        #Check if [gate,inp] already in the circuit:
        if lasgate.count([gate,inp])!=arg:
            #print([gate,inp],arg)
            for i in inp:
                lasgate[i]=[gate,inp]
            gate(*inp)

#        try: counts[gate] +=1
#        except KeyError: counts[gate] = 1

            hcounter+=int(gate==qc.h)

        if qc.depth() == depth+1:
            hcounter+=(-int(gate==qc.h))
            del(qc.data[-1])
            break

        if hcounter==hgates:
            if list_of_gates[0]==(qc.h,1):
                del(list_of_gates[0])


        #print(qc.depth())
    #for i in lasgate:
    #    print(i)
    return qc,hcounter



In [ ]:
qc,hg = rnd_cg(7,10)
print(hg)

5


In [ ]:
qc.draw()

q_0: ────■─────────────■────────────────────■────────■────
         │             │              ┌───┐ │        │    
q_1: ─■──┼───────■──■──┼────────────■─┤ Z ├─┼─────■──┼──■─
      │  │ ┌───┐ │  │  │            │ └───┘ │     │  │  │ 
q_2: ─┼──┼─┤ H ├─┼──┼──┼────────────┼───────┼─────┼──■──┼─
      │  │ └───┘ │  │  │ ┌───┐┌───┐ │       │     │  │  │ 
q_3: ─■──┼───────┼──■──┼─┤ Z ├┤ H ├─┼───────■──■──■──┼──■─
         │ ┌───┐ │  │  │ ├───┤└───┘ │ ┌───┐    │  │  │    
q_4: ────■─┤ H ├─■──┼──■─┤ Z ├──────■─┤ H ├────■──┼──■────
         │ └───┘ │  │    └───┘        └───┘       │       
q_5: ────■───────■──■─────────────────────────────┼───────
           ┌───┐                                  │       
q_6: ──────┤ H ├──────────────────────────────────■───────
           └───┘

# Analysis of the random circuits:

Now to perform an analysis, we use random circuits formed from various parameters and keep track of

1. (parameter) Width 'n'
2. (parameter) Depth 'd'
3. (parameter) No. of hadamard gates 'h'
4. Simulation Runtime

We do the sampling 2 (cuz 3 was giving a complication) times, each time effectively generating data for a surface plot for two of the parameters v/s simulation runtime. I have modified the original function to just save the result into an object instead of printing it, since as of now our simulation tools are capableof simulating circuits upto 10-11 qubits at a decent speed. Have removed all print statements to prevent crashes occuring due to excessive printing.



In [ ]:
import pandas as pd
from time import time
from qiskit import qasm3

"""
The following fields go into table
    QC = generated quantum circuit
    n = Width
    d = Depth
    h = Hadamard Count
    R = runtime
"""

'\nThe following fields go into table\n    QC = generated quantum circuit\n    n = Width\n    d = Depth\n    h = Hadamard Count\n    R = runtime\n'

In [ ]:
#Part 1: n vs h at a given d: We fix d at some depths and obtain n - h - runtime data

df_fixed_d = pd.DataFrame()

#We will iterate d = 1 to 6 and in each iteration we generate 1000 (will change value) of random circuits for random input values of n.
#I am not setting any bar on the number of hadamards for now, and I will allow the width to vary randomly between 1 to 6

for d in range(1,7):
    for i in range(1000):
        #Step 1: Choose a width
        width = randrange(1,7)
        #Step 2: Generate Random Circuit
        qc,hg = rnd_cg(width,d)
        #Step 3: Compute computation time of simulation of result
        start_time = time()
        simulate_viapoly(qc)#not storing it anywhere as of now
        runtime = time() - start_time
        #Step 4: Add record in our dataframe
        recd = {"QC":[(instruction.operation.name,[qc.find_bit(q).index for q in instruction.qubits]) for index, instruction in enumerate(qc.data)],
                "Width(n)":width,
                "Depth":d,
                "Hadamard Counts":hg,
                "Runtime":runtime}
        df_fixed_d = df_fixed_d._append(recd,ignore_index=True)

print(df_fixed_d)

#Part 2: now we do d vs h at a given h. We fix n at some number of qubits and obtain d - h - runtime data,

df_fixed_n = pd.DataFrame()

#We will iterate n = 1 to 6 and in each iteration we generate 1000 (will change value) of random circuits for random input values of n.
#I am not setting any bar on the number of hadamards for now, and I will allow the width to vary randomly between 1 to 6


for n in range(1,7):
    for i in range(1000):
        #Step 1: Choose a depth
        d = randrange(1,7)
        #Step 2: Generate Random Circuit
        qc,hg = rnd_cg(n,d)
        #Step 3: Compute computation time of simulation of result
        start_time = time()
        simulate_viapoly(qc)#not storing it anywhere as of now
        runtime = time() - start_time
        #Step 4: Add record in our dataframe
        recd = {"QC":[(instruction.operation.name,[qc.find_bit(q).index for q in instruction.qubits]) for index, instruction in enumerate(qc.data)],#qasm3.dumps(qc),
                "Width(n)":width,
                "Depth":d,
                "Hadamard Counts":hg,
                "Runtime":runtime}
        df_fixed_n = df_fixed_n._append(recd,ignore_index=True)


print(df_fixed_n)


                                                     QC  Width(n)  Depth  \
0                                            [(h, [0])]         1      1   
1                                            [(h, [0])]         1      1   
2                                            [(h, [0])]         1      1   
3                                            [(z, [0])]         1      1   
4                              [(cz, [2, 3]), (h, [0])]         5      1   
...                                                 ...       ...    ...   
5995  [(z, [0]), (h, [0]), (z, [0]), (h, [0]), (z, [...         1      6   
5996  [(z, [1]), (cz, [0, 1]), (z, [1]), (h, [0]), (...         2      6   
5997  [(z, [1]), (z, [3]), (h, [4]), (cz, [4, 5]), (...         6      6   
5998  [(h, [0]), (h, [1]), (cz, [0, 1]), (h, [1]), (...         2      6   
5999  [(h, [0]), (h, [1]), (z, [0]), (cz, [0, 1]), (...         2      6   

      Hadamard Counts   Runtime  
0                   1  0.000732  
1                  

In [ ]:
from time import time,sleep
start = time()

time() - start

3.0994415283203125e-05

In [ ]:
for i in range(10):
    i

In [ ]:
df_fixed_d._append(recd,ignore_index=True)
print(df_fixed_d)

Empty DataFrame
Columns: []
Index: []


In [ ]:
recd

{'QC': 'OPENQASM 3.0;\ninclude "stdgates.inc";\ngate ccz _gate_q_0, _gate_q_1, _gate_q_2 {\n  h _gate_q_2;\n  ccx _gate_q_0, _gate_q_1, _gate_q_2;\n  h _gate_q_2;\n}\nqubit[5] q;\nz q[1];\ncz q[2], q[4];\ncz q[3], q[4];\ncz q[2], q[3];\ncz q[0], q[4];\ncz q[0], q[2];\ncz q[2], q[3];\nh q[3];\ncz q[1], q[4];\nh q[0];\nccz q[0], q[1], q[4];\nz q[2];\n',
 'Width(n)': 5,
 'Depth': 6,
 'Hadamard Counts': 2,
 'Runtime': 2.5464680194854736}

In [ ]:
[(instruction.operation.name,[qc.find_bit(q).index for q in instruction.qubits]) for index, instruction in enumerate(qc.data)]

[('cz', [1, 3]),
 ('ccz', [0, 4, 5]),
 ('h', [4]),
 ('ccz', [1, 4, 5]),
 ('ccz', [1, 3, 5]),
 ('cz', [0, 4]),
 ('z', [4]),
 ('z', [3]),
 ('h', [3]),
 ('h', [2]),
 ('cz', [1, 4]),
 ('h', [6]),
 ('h', [4]),
 ('z', [1]),
 ('cz', [0, 3]),
 ('cz', [3, 4]),
 ('ccz', [1, 3, 6]),
 ('cz', [1, 3]),
 ('ccz', [0, 2, 4])]

In [ ]:
qc.draw()

┌───┐
q_0: ────■──■──■─┤ Z ├
         │  │  │ └───┘
q_1: ────┼──┼──■──────
         │  │  │      
q_2: ─■──┼──┼──┼──────
      │  │  │  │      
q_3: ─■──■──┼──┼──────
         │  │  │      
q_4: ────┼──┼──┼──────
         │  │  │      
q_5: ────■──■──■──────

In [ ]:
from qiskit import QuantumCircuit

# Create your quantum circuit
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.z(1)

# Get the list of gates in the order they are applied
gate_list = qc.data

# Print the gates
for gate in gate_list:
    print(gate)


CircuitInstruction(operation=Instruction(name='h', num_qubits=1, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(2, 'q'), 0),), clbits=())
CircuitInstruction(operation=Instruction(name='cx', num_qubits=2, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(2, 'q'), 0), Qubit(QuantumRegister(2, 'q'), 1)), clbits=())
CircuitInstruction(operation=Instruction(name='z', num_qubits=1, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(2, 'q'), 1),), clbits=())


In [ ]:
import qiskit

In [ ]:
import inspect

In [ ]:
print(inspect.getsource(qiskit.circuit.quantumregister.Qubit))

class Qubit(Bit):
    """Implement a quantum bit."""

    __slots__ = ()

    def __init__(self, register=None, index=None):
        """Creates a qubit.

        Args:
            register (QuantumRegister): Optional. A quantum register containing the bit.
            index (int): Optional. The index of the bit in its containing register.

        Raises:
            CircuitError: if the provided register is not a valid :class:`QuantumRegister`
        """

        if register is None or isinstance(register, QuantumRegister):
            super().__init__(register, index)
        else:
            raise CircuitError(
                "Qubit needs a QuantumRegister and %s was provided" % type(register).__name__
            )



In [ ]:
for gate in gate_list:
    instruction, qargs, cargs = gate
    print(type(qargs))
    print(f"Gate: {instruction.name}, Qubits: {[qc.find_bit(q).index for q in qargs]}, Clbits: {[clbit.index for clbit in cargs]}")


<class 'list'>
Gate: h, Qubits: [0], Clbits: []
<class 'list'>
Gate: cx, Qubits: [0, 1], Clbits: []
<class 'list'>
Gate: z, Qubits: [1], Clbits: []
